Founders and Financials: Machine Learning Algorithms in Venture Capital by Viktor Lado Naess and Emrik Stål

Viktor Lado Naess: viktor.ladonaess@gmail.com;  +46706719459

Emrik Stål: emrik02@gmail.com; +46735931454

In [ ]:
#importing data from sheet
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

spreadsheet = gc.open('final data')

worksheet = spreadsheet.worksheet('Sheet1')


data = worksheet.get_all_values()

#Assiging correct data types
import pandas as pd
df = pd.DataFrame(data[1:], columns=data[0])
import numpy as np
df.loc[:,'More than one round'] = df['More than one round'].astype(int)
df['Share female founders'] = df['Share female founders'].replace('', np.nan) #We do not intend to filter this so assigning nan instead of ''
df.loc[:,'Share female founders'] = df['Share female founders'].astype(float)
df.loc[:,'Distance from Stockholm'] = df['Distance from Stockholm'].astype(float)
df.loc[:,'Serial Founder'] = df['Serial Founder'].astype(int)
df.loc[:,'Branch Sector'] = df['Branch Sector'].astype(int)
df.loc[:,'Quick Ratio'] = df['Quick Ratio'].astype(float)
df.loc[:,'Return on Equity'] = df['Return on Equity'].astype(float)
df.loc[:,'Net_sales'] = df['Net_sales'].astype(float)
df.loc[:,'Net Sales Ratio'] = df['Net Sales Ratio'].astype(float)
rf_cols = ['More than one round', 'Distance from Stockholm', 'Serial Founder', 'Quick Ratio', 'Return on Equity', 'Net Sales Ratio','Share female founders']
df=df[rf_cols]

random_seed =8

#Removing outliers

In [ ]:
from scipy import stats

# List of columns to iterate through
columns = ['Return on Equity', 'Quick Ratio', 'Net Sales Ratio']


outlier_indices = set()

for col in columns:
    # Calculate Z-scores for the column
    z_scores = stats.zscore(df[col])

    # Find indices of outliers (where Z-score absolute value is greater than 3)
    outliers = df[abs(z_scores) > 3].index

    # Add outlier indices to the set
    outlier_indices.update(outliers)

# Drop the outlier rows from the DataFrame
df = df.drop(index=outlier_indices)

# df_cleaned is now the DataFrame without the outliers
df

In [ ]:
df['More than one round'].value_counts()

#Random forest

The following section of code is highly intensive for hardware. If decided to run on laptop, make sure all other programmes and tabs are closed and be prepared to do manual restart for google(if on colab) or other running software.


It is understood that not all who run this may have the specifications to complete the run (at its fastest it will be 1.5 hours, but commonly between 2-5 hours). As such an simpler version is provided below, which is not intensive. This version  is slightly less accurate however resutls only differ minorly.

In [ ]:

# !pip install optuna
# import pandas as pd
# import optuna
# import numpy as np
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, brier_score_loss, roc_curve, roc_auc_score
# from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
# from sklearn.utils import resample
# from sklearn.model_selection import KFold


# y = df['More than one round']
# X = df.drop(columns=['More than one round', 'Share female founders'])
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_seed, stratify=y)

# parameter_ranges = [
#     {'n_estimators_range': (50, 200), 'max_depth_range': (1, 5)},
#     {'n_estimators_range': (50, 200), 'max_depth_range': (5, 10)},
#     {'n_estimators_range': (50, 200), 'max_depth_range': (10, 15)},
#     {'n_estimators_range': (200, 550), 'max_depth_range': (1, 5)},
#     {'n_estimators_range': (200, 550), 'max_depth_range': (5, 10)},
#     {'n_estimators_range': (200, 550), 'max_depth_range': (10, 15)},
# ]

# results1 = []
# random_seed = 8
# for param_range in parameter_ranges:
#     def objective(trial):

#         n_estimators = trial.suggest_int('n_estimators', param_range['n_estimators_range'][0], param_range['n_estimators_range'][1])
#         max_depth = trial.suggest_int('max_depth', param_range['max_depth_range'][0], param_range['max_depth_range'][1])
#         min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
#         min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)
#         max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2'])
#         # strat_k_fold =trial.suggest_categorical('strat_k_fold', [7,StratifiedKFold(n_splits=7)])

#         rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,
#                                     min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf,
#                                     max_features=max_features, random_state=random_seed)


#         rf.fit(X_train, y_train)
#         y_pred_best = rf.predict(X_train)
#         accuracy_best = accuracy_score(y_train, y_pred_best)


#         #Cross validation training accuracy
#         strat_k_fold=StratifiedKFold(n_splits=7, shuffle=True, random_state=random_seed)
#         crossAcc = cross_val_score(rf, X_train, y_train, cv=strat_k_fold, n_jobs=-1).mean()


#         return accuracy_best, crossAcc


#     sampler = optuna.samplers.TPESampler(seed=random_seed)
#     study = optuna.create_study(directions=['maximize', 'maximize'], sampler=sampler)
#     study.optimize(objective, n_trials=300)

#     for trial in study.best_trials:

#             best_rf = RandomForestClassifier(**trial.params)
#             best_rf.fit(X_train, y_train)

#             # Additional metrics calculations
#             cv_splitter = KFold(n_splits=7, shuffle=True, random_state=random_seed)


#             y_pred_best = best_rf.predict(X_test)
#             accuracy_best = accuracy_score(y_test, y_pred_best)
#             precision_best = precision_score(y_test, y_pred_best)
#             recall_best = recall_score(y_test, y_pred_best)
#             f1_best = f1_score(y_test, y_pred_best)
#             cross_val_best_rf = np.mean(cross_val_score(best_rf, X_train, y_train, cv=cv_splitter))
#             cross_val_best_rf_test = np.mean(cross_val_score(best_rf, X_test, y_test, cv=cv_splitter))
#             cross_val_best_rf_all = np.mean(cross_val_score(best_rf, X, y, cv=cv_splitter))
#             y_pred_probs_best = best_rf.predict_proba(X_test)[:, 1]
#             brier_score_best = brier_score_loss(y_test, y_pred_probs_best)
#             roc_auc_best = roc_auc_score(y_test, y_pred_probs_best)

#             # Bootstrap scores
#             rf_bootstrap_scores = []
#             n_bootstrap = 100
#             for seed in range(8, 8 + n_bootstrap):
#                 np.random.seed(seed)
#                 X_test_sample, y_test_sample = resample(X_test, y_test)
#                 y_pred_sample = best_rf.predict(X_test_sample)
#                 score = accuracy_score(y_test_sample, y_pred_sample)
#                 rf_bootstrap_scores.append(score)
#             rf_average_score = np.mean(rf_bootstrap_scores)
#             rf_variance_score = np.var(rf_bootstrap_scores)

#             #placebo
#             np.random.seed(random_seed)
#             X_train_random = np.random.random(size=X_train.shape)
#             rf_random = RandomForestClassifier(**trial.params)
#             rf_random.fit(X_train_random, y_train)
#             y_pred_random = rf_random.predict(X_test)
#             accuracy_random = accuracy_score(y_test, y_pred_random)

#             # Append results to the list
#             results1.append({
#                 'Parameter Range': param_range,
#                 'Best Hyperparameters': trial.params,
#                 'Accuracy': accuracy_best,
#                 'Cross Validation test Accuracy':cross_val_best_rf_test,
#                 'Bootstrap Average Score': rf_average_score,
#                 'Placebo Accuracy': accuracy_random,
#                 'Cross Validation train Accuracy': cross_val_best_rf,
#                 'Precision': precision_best,
#                 'Recall': recall_best,
#                 'F1 Score': f1_best,
#                 'Brier Score': brier_score_best,
#                 'ROC AUC Score': roc_auc_best,
#                 'Bootstrap Variance Score': rf_variance_score
#             })

# # Convert results list to DataFrame
# results_df1 = pd.DataFrame(results1)
# results_df1


In [ ]:
# results_df1 = pd.DataFrame(results1)
# results_df1


In [ ]:
# results_df1['Sum'] = results_df1['Accuracy'] + results_df1['Cross Validation test Accuracy'] + results_df1['Bootstrap Average Score'] -results_df1['Placebo Accuracy']
# results_df1['Processed_Min'] = results_df1[['Accuracy', 'Cross Validation test Accuracy', 'Bootstrap Average Score']].min(axis=1) - results_df1['Placebo Accuracy']

# normalized_df = (results_df1 - results_df1.min()) / (results_df1.max() - results_df1.min())
# results_df1['sum norm'] = normalized_df['Sum']
# results_df1['process norm'] = normalized_df['Processed_Min']

# results_df1['final vals'] = results_df1['sum norm'] + results_df1['process norm']
# pd.set_option('display.max_rows', None)  # Show all rows
# pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)
# results_df1

In [ ]:
# max_final_vals_index = results_df1['final vals'].idxmax()

# # Extract the row with the highest 'final vals'
# highest_val = results_df1.loc[max_final_vals_index]
# # highest_val

#rf analysis

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, brier_score_loss, roc_curve, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.utils import resample
from sklearn.model_selection import KFold

# best_hyperparams = highest_val['Best Hyperparameters']
# best_rf = RandomForestClassifier(**best_hyperparams)

#Use the below code if the Optimization takes too long. Use the above code if the optimizaiton code is run. Be sure to uncomment the non used code in either case

best_rf= RandomForestClassifier(max_depth=2, max_features='auto', min_samples_leaf=3, min_samples_split=18, n_estimators=94)
y = df['More than one round']
X = df.drop(columns=['More than one round', 'Share female founders'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_seed, stratify=y)
best_rf.fit(X_train, y_train)

In [ ]:
#Creating a confusion matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
y_pred = best_rf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap='Blues')
disp.ax_.grid(False)
plt.show()

In [ ]:
#feature importance weights
feature_importances = pd.Series(best_rf.feature_importances_, index=X_train.columns).sort_values(ascending=False)
feature_importances

In [ ]:
#Bar plot of importances
feature_importances.plot.bar();

In [ ]:
#creating partial depeendence plots
from sklearn.inspection import PartialDependenceDisplay

PartialDependenceDisplay.from_estimator(best_rf, X_train, features=['Distance from Stockholm'], kind = "average")
PartialDependenceDisplay.from_estimator(best_rf, X_train, features=['Serial Founder'], kind = "average")
PartialDependenceDisplay.from_estimator(best_rf, X_train, features=['Quick Ratio'], kind = "average")
PartialDependenceDisplay.from_estimator(best_rf, X_train, features=['Return on Equity'], kind = "average")
PartialDependenceDisplay.from_estimator(best_rf, X_train, features=['Net Sales Ratio'], kind = "average")


In [ ]:
#installing shap package and producing shap summary plots
!pip install shap
import shap
explainer = shap.TreeExplainer(best_rf)
shap_values = explainer.shap_values(X_train)
shap.summary_plot(shap_values, X_train)

In [ ]:
#Scatter Shap summery plot
shap.summary_plot(shap_values[1], X_train)

In [ ]:
#Shap decision plot- ultimately not used in paper.
shap.decision_plot(explainer.expected_value[1], shap_values[1], X_test.columns)

In [ ]:
#Partial depende plots for SHAP Values
for i in range(len(shap_values)):

    for feature_index in range(X_train.shape[1]):
        shap.dependence_plot(feature_index, shap_values[1], X_train, interaction_index=None)

#compare w logit

In [ ]:
#builindg a logistic model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, brier_score_loss, roc_auc_score
from sklearn.model_selection import cross_val_score, KFold, train_test_split, StratifiedKFold
from sklearn.utils import resample
import numpy as np
import pandas as pd


logres = []
# Create a Logistic Regression model
best_logit = LogisticRegression(max_iter=1000)
best_logit.fit(X_train, y_train)

# Additional metrics calculations
cv_splitter = KFold(n_splits=7, shuffle=True, random_state=random_seed)

y_pred_best = best_logit.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)
precision_best = precision_score(y_test, y_pred_best)
recall_best = recall_score(y_test, y_pred_best)
f1_best = f1_score(y_test, y_pred_best)
cross_val_best_logit = np.mean(cross_val_score(best_logit, X_train, y_train, cv=cv_splitter))
cross_val_best_logit_test = np.mean(cross_val_score(best_logit, X_test, y_test, cv=cv_splitter))
cross_val_best_logit_all = np.mean(cross_val_score(best_logit, X, y, cv=cv_splitter))
y_pred_probs_best = best_logit.predict_proba(X_test)[:, 1]
brier_score_best = brier_score_loss(y_test, y_pred_probs_best)
roc_auc_best = roc_auc_score(y_test, y_pred_probs_best)

# Bootstrap scores
logit_bootstrap_scores = []
n_bootstrap = 100
for seed in range(8, 8 + n_bootstrap):
    np.random.seed(seed)
    X_test_sample, y_test_sample = resample(X_test, y_test)
    y_pred_sample = best_logit.predict(X_test_sample)
    score = accuracy_score(y_test_sample, y_pred_sample)
    logit_bootstrap_scores.append(score)
logit_average_score = np.mean(logit_bootstrap_scores)
logit_variance_score = np.var(logit_bootstrap_scores)

# Placebo test
np.random.seed(random_seed)
X_train_random = np.random.random(size=X_train.shape)
logit_random = LogisticRegression()
logit_random.fit(X_train_random, y_train)
y_pred_random = logit_random.predict(X_test)
accuracy_random = accuracy_score(y_test, y_pred_random)

logres.append({
    'Model': 'Logit',
    'Accuracy': accuracy_best,
    'Cross Validation test Accuracy': cross_val_best_logit_test,
    'Bootstrap Average Score': logit_average_score,
    'Placebo Accuracy': accuracy_random,
    'Cross Validation train Accuracy': cross_val_best_logit,
    'Precision': precision_best,
    'Recall': recall_best,
    'F1 Score': f1_best,
    'Brier Score': brier_score_best,
    'ROC AUC Score': roc_auc_best,
    'Bootstrap Variance Score': logit_variance_score
})

# Convert logres list to DataFrame
logres_df = pd.DataFrame(logres)
logres_df


In [ ]:
# columns_to_drop = ['Parameter Range', 'Best Hyperparameters', 'Sum', 'Processed_Min', 'sum norm', 'process norm','final vals']

# # Drop the specified columns
# highest_val['Model'] = 'Random forest'
# clean_highest_val = highest_val.drop(columns_to_drop)
# logres_df = logres_df.append(clean_highest_val, ignore_index=True)


#The below code can be used if The optimization code is not run or does not complete its run. If the optimization code is run be sure to sure uncomment the above code and comment the below code.

best_rf.fit(X_train, y_train)

# Additional metrics calculations
cv_splitter = KFold(n_splits=7, shuffle=True, random_state=random_seed)

y_pred_best = best_rf.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)
precision_best = precision_score(y_test, y_pred_best)
recall_best = recall_score(y_test, y_pred_best)
f1_best = f1_score(y_test, y_pred_best)
cross_val_best_rf = np.mean(cross_val_score(best_rf, X_train, y_train, cv=cv_splitter))
cross_val_best_rf_test = np.mean(cross_val_score(best_rf, X_test, y_test, cv=cv_splitter))
cross_val_best_rf_all = np.mean(cross_val_score(best_rf, X, y, cv=cv_splitter))
y_pred_probs_best = best_rf.predict_proba(X_test)[:, 1]
brier_score_best = brier_score_loss(y_test, y_pred_probs_best)
roc_auc_best = roc_auc_score(y_test, y_pred_probs_best)

# Bootstrap scores
rf_bootstrap_scores = []
n_bootstrap = 100
for seed in range(8, 8 + n_bootstrap):
    np.random.seed(seed)
    X_test_sample, y_test_sample = resample(X_test, y_test)
    y_pred_sample = best_rf.predict(X_test_sample)
    score = accuracy_score(y_test_sample, y_pred_sample)
    rf_bootstrap_scores.append(score)
rf_average_score = np.mean(rf_bootstrap_scores)
rf_variance_score = np.var(rf_bootstrap_scores)

# Placebo test
np.random.seed(random_seed)
X_train_random = np.random.random(size=X_train.shape)
rf_random = LogisticRegression()
rf_random.fit(X_train_random, y_train)
y_pred_random = rf_random.predict(X_test)
accuracy_random = accuracy_score(y_test, y_pred_random)

logres.append({
    'Model': 'rf',
    'Accuracy': accuracy_best,
    'Cross Validation test Accuracy': cross_val_best_rf_test,
    'Bootstrap Average Score': rf_average_score,
    'Placebo Accuracy': accuracy_random,
    'Cross Validation train Accuracy': cross_val_best_rf,
    'Precision': precision_best,
    'Recall': recall_best,
    'F1 Score': f1_best,
    'Brier Score': brier_score_best,
    'ROC AUC Score': roc_auc_best,
    'Bootstrap Variance Score': rf_variance_score
})

# Convert logres list to DataFrame
logres_df = pd.DataFrame(logres)
logres_df


In [ ]:
logres_df

In [ ]:
#ROC and AUC plots
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

# Predict probabilities
y_pred_probs_rf = best_rf.predict_proba(X_test)[:, 1]  # Probabilities for the positive class
y_pred_probs_logit = best_logit.predict_proba(X_test)[:, 1]

# Compute ROC curve and ROC area for each model
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_pred_probs_rf)
roc_auc_rf = auc(fpr_rf, tpr_rf)

fpr_logit, tpr_logit, _ = roc_curve(y_test, y_pred_probs_logit)
roc_auc_logit = auc(fpr_logit, tpr_logit)

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(fpr_rf, tpr_rf, color='darkgreen', lw=2, label=f'Random Forest (area = {roc_auc_rf:.2f})')
plt.plot(fpr_logit, tpr_logit, color='blue', lw=2, label=f'Logistic Regression (area = {roc_auc_logit:.2f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC and AUC')
plt.legend()
plt.grid(True)
plt


In [ ]:
# Precision Recall curve and graph
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
y_scores_best_logit = best_logit.predict_proba(X_test)[:, 1]
y_scores_best_rf = best_rf.predict_proba(X_test)[:, 1]



precision_best_logit, recall_best_logit, thresholds_best_logit = precision_recall_curve(y_test, y_scores_best_logit)
precision_best_rf, recall_best_rf, thresholds_best_rf = precision_recall_curve(y_test, y_scores_best_rf)


plt.figure(figsize=(10, 7))
plt.plot(recall_best_logit, precision_best_logit, label='Logit', color='blue')
plt.plot(recall_best_rf, precision_best_rf, label='Random Forest', color='darkgreen')



plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.grid(True)
plt.show